In [1]:
import sys
import os
import torch
import matplotlib.pyplot as plt
import math
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_mll
from botorch.optim import optimize_acqf
from botorch.acquisition import ExpectedImprovement, LogExpectedImprovement, PosteriorMean
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.utils.transforms import unnormalize, normalize
from pprint import pprint

from src.scheffe_generator import ScheffeGenerator

# 設定設備與型別
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.double

/home/appuser/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset_path = '/workspaces/BO_EXPERIMENTS/src/datasets/D=3_N=5/oracle_data_D3_B_003.pt'

dataset = torch.load(dataset_path)

In [3]:
# 設定 Oracle Function
variant = dataset['config']['variant']
D = dataset['config']['D']
seed = dataset['config']['seed']
gen = ScheffeGenerator( D=D, variant=variant, seed=seed )

# initial dataset
train_x = dataset['initial_data']['X'].to(device)
train_obj = dataset['initial_data']['Y'].to(device)
gt_x = dataset['ground_truth']['x_star'].to(device)
gt_y = dataset['ground_truth']['f_star']

 # Set X bound
bounds = torch.stack([torch.zeros(D), torch.ones(D)]).to(device, dtype=dtype)

# Set constraints
constraints = [
    (torch.arange(D, device=device), torch.ones(D, dtype=dtype, device=device), 1.0)
]

# # 優化過程
# result = BO_with_GP_EI_and_SLSQP(
#     train_x, train_obj, gt_x, gt_y, gen.oracle, bounds, constraints, n_iterations
# )

n_iterations = 20
gt_func = gen.oracle

In [4]:
inference_regrets = [] # 用來存 inference regret 的數值
simple_regrets = [] # 用來儲存 simple regret 的數值

for i in range(n_iterations):
    # Fit GP surrogate model
    gp = SingleTaskGP(train_x, train_obj)
    mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
    fit_gpytorch_mll(mll)

    # Find Surrogate Model Best Point
    post_mean_func = PosteriorMean(model=gp)
    best_predicted_x, _ = optimize_acqf(
        acq_function=post_mean_func,
        bounds=bounds,
        q=1,
        num_restarts=10,
        raw_samples=100,
        equality_constraints=constraints,
    )

    # Get New Observe Data
    new_y = gt_func( X=best_predicted_x.cpu().numpy(), noiseless=True)

    # Set Acquisition Function
    best_f = train_obj.max().item()
    EI = LogExpectedImprovement(model=gp, best_f=best_f)

    # Optimize
    candidate, acq_value = optimize_acqf(
        acq_function=EI,
        bounds=bounds,
        q=1,                     # 每次推薦 1 個點
        num_restarts=10,         # 隨機重啟次數（類似 SLSQP 的重啟）
        raw_samples=100,         # 初始採樣點數量
        equality_constraints=constraints,
    )

    # Combine Old and New Observe Data
    train_x = torch.cat([train_x, candidate])
    train_obj = torch.cat([train_obj, torch.tensor(new_y, device=device).unsqueeze(0) ])
    
    # Compute Simple Regret
    max_train_obj = train_obj.max().item()
    simple_regret = gt_y - max_train_obj
    simple_regrets.append(simple_regret)

    # Compute Inference Regrets
    infer_regret = float((gt_y - new_y)[0])
    inference_regrets.append(infer_regret)
    
    print(f'Epoch {i+1}: Real Best Value: {gt_y:.2f}, Max Train Obj = {max_train_obj:.2f}, Current Train Obj = {float(new_y[0]):.2f}, Simple Regret = {simple_regret:.2f}, Infer Regret = {infer_regret:.2f}, SumX = {candidate.sum().item():.1f}')

output = {
    'inference_regrets': inference_regrets,
    'simple_regrets': simple_regrets,
    'opt_x': candidate.cpu().numpy().tolist(),
    'opt_y': float(new_y[0])
}

Epoch 1: Real Best Value: 1.44, Max Train Obj = 1.40, Current Train Obj = 1.40, Simple Regret = 0.05, Infer Regret = 0.05, SumX = 1.0
Epoch 2: Real Best Value: 1.44, Max Train Obj = 1.41, Current Train Obj = 1.41, Simple Regret = 0.03, Infer Regret = 0.03, SumX = 1.0
Epoch 3: Real Best Value: 1.44, Max Train Obj = 1.42, Current Train Obj = 1.42, Simple Regret = 0.02, Infer Regret = 0.02, SumX = 1.0
Epoch 4: Real Best Value: 1.44, Max Train Obj = 1.42, Current Train Obj = 1.41, Simple Regret = 0.02, Infer Regret = 0.04, SumX = 1.0
Epoch 5: Real Best Value: 1.44, Max Train Obj = 1.42, Current Train Obj = 1.35, Simple Regret = 0.02, Infer Regret = 0.09, SumX = 1.0
Epoch 6: Real Best Value: 1.44, Max Train Obj = 1.44, Current Train Obj = 1.44, Simple Regret = 0.00, Infer Regret = 0.00, SumX = 1.0
Epoch 7: Real Best Value: 1.44, Max Train Obj = 1.44, Current Train Obj = 1.44, Simple Regret = 0.00, Infer Regret = 0.00, SumX = 1.0
Epoch 8: Real Best Value: 1.44, Max Train Obj = 1.44, Current 

In [19]:
test_tensor = torch.tensor([[100,100,100]], dtype=torch.double).to(device)
with torch.no_grad():
    # 取得後驗分佈
    posterior = gp.posterior(test_tensor)
    
    # 取得預測平均值 (Mean)
    mean = posterior.mean
    
    # 取得預測變異數 (Variance) 或 標準差 (Stddev)
    variance = posterior.variance
    stddev = torch.sqrt(variance)

print(mean, stddev)

tensor([[0.7780]], device='cuda:0', dtype=torch.float64) tensor([[0.2239]], device='cuda:0', dtype=torch.float64)


In [12]:
mean

tensor([[0.7780]], device='cuda:0', dtype=torch.float64)